# Explore Kubeflow Katib Hyperparamter Optimization API

In [1]:
import sys
print(sys.version)

3.12.8 (main, Dec  3 2024, 18:42:41) [Clang 16.0.0 (clang-1600.0.26.4)]


In [17]:
!kubectl apply -k "github.com/kubeflow/katib.git/manifests/v1beta1/installs/katib-standalone?ref=master"

namespace/kubeflow configured
customresourcedefinition.apiextensions.k8s.io/experiments.kubeflow.org created
customresourcedefinition.apiextensions.k8s.io/suggestions.kubeflow.org created
customresourcedefinition.apiextensions.k8s.io/trials.kubeflow.org created
serviceaccount/katib-controller created
serviceaccount/katib-ui created
clusterrole.rbac.authorization.k8s.io/katib-controller created
clusterrole.rbac.authorization.k8s.io/katib-ui created
clusterrolebinding.rbac.authorization.k8s.io/katib-controller created
clusterrolebinding.rbac.authorization.k8s.io/katib-ui created
configmap/katib-config created
configmap/trial-templates created
secret/katib-mysql-secrets created
secret/katib-webhook-cert created
service/katib-controller created
service/katib-db-manager created
service/katib-mysql created
service/katib-ui created
persistentvolumeclaim/katib-mysql created
deployment.apps/katib-controller created
deployment.apps/katib-db-manager created
deployment.apps/katib-mysql created
dep

In [3]:
%pip install git+https://github.com/kubeflow/katib.git#subdirectory=sdk/python/v1beta1 --break-system-packages
%pip install git+https://github.com/kubeflow/trainer.git@pr-2576#egg=trainer-sdk&subdirectory=sdk/python --break-system-packages

  Cloning https://github.com/kubeflow/katib.git to /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-kz6279aq
  Running command git clone --filter=blob:none --quiet https://github.com/kubeflow/katib.git /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-kz6279aq
  Resolved https://github.com/kubeflow/katib.git to commit dd4acfc2ce8fb4e4e0095a49d012443e26c70b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached setuptools-79.0.1-py3-none-any.whl.metadata (6.5 kB)
  Using cached kubeflow_training-1.9.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached retrying-1.3.4-py3-none-any.whl.metadata (6.9 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests-2.32.3-py

In [7]:
# i assume after the package updates in this pr, i don't need to update them manually
%pip install transformers peft --break-system-packages

  Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 18.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.3 MB/s eta 0:00:00a 0:00:01
Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl (284 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 21.1 MB/s eta 0:00:0000:0100:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
 

In [20]:
from kubeflow.katib import TrainerResources


resources_per_trial = TrainerResources(
    num_workers=1,                    # Number of distributed workers
    num_procs_per_worker=1,           # Processes per worker
    resources_per_worker={            # Resource allocation per worker
        "gpu": 0,                     # Number of GPUs
        "cpu": 1,                     # Number of CPUs
        "memory": "1G",              # Memory allocation
    },
)

In [8]:
import kubeflow.katib as katib
from kubeflow.katib import KatibClient

from transformers import AutoModelForSequenceClassification, TrainingArguments
from peft import LoraConfig

from kubeflow.storage_initializer.hugging_face import (
    HuggingFaceModelParams,
    HuggingFaceDatasetParams,
    HuggingFaceTrainerParams,
)

/opt/homebrew/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
hf_model = HuggingFaceModelParams(
    model_uri = "hf://meta-llama/Llama-3.2-1B",
    transformer_type = AutoModelForSequenceClassification,
)

# Train the model on 1 movie reviews from imdb
# https://huggingface.co/datasets/stanfordnlp/imdb
hf_dataset = HuggingFaceDatasetParams(
    repo_id = "imdb",
    split = "train[:1]",
)

hf_tuning_parameters = HuggingFaceTrainerParams(
    training_parameters = TrainingArguments(
        output_dir = "results",
        save_strategy = "no",
        hub_strategy="all_checkpoints",
        learning_rate = 1e-05, #katib.search.double(min=1e-05, max=5e-05),
        num_train_epochs=1,
    ),
    lora_config=LoraConfig(
        r = 8,
        lora_alpha=8,
        lora_dropout=0.1,
        bias="none",
    )
)

In [16]:
cl = KatibClient(namespace="kubeflow")

In [25]:
# Fine-tuning for Binary Classification
exp_name = "testllm"
cl.tune(
    name = exp_name,
    model_provider_parameters = hf_model,
    dataset_provider_parameters = hf_dataset,
    trainer_parameters = hf_tuning_parameters,
    objective_metric_name = "train_loss",
    objective_type = "minimize",
    algorithm_name = "random",
    max_trial_count = 10,
    parallel_trial_count = 2,
    resources_per_trial=resources_per_trial,
)

cl.wait_for_experiment_condition(name=exp_name)

# Get the best hyperparameters.
print(cl.get_optimal_hyperparameters(exp_name))

Thank you for using `tune` API for LLM hyperparameter optimization. This feature is in the alpha stage. Kubeflow community is looking for your feedback. Please share your experience via #kubeflow-katib Slack channel or the Kubeflow Katib GitHub.
PVC 'testllm' already exists in namespace kubeflow.


TypeError: Object of type LoraRuntimeConfig is not JSON serializable